In [ ]:
# Install necessary libraries
!pip install gradio pandas textblob wordcloud matplotlib nltk scikit-learn sumy

# Download necessary NLTK data
import nltk
nltk.download('stopwords')
nltk.download('punkt')


In [ ]:
import gradio as gr
import pandas as pd
from textblob import TextBlob
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from sklearn.feature_extraction.text import TfidfVectorizer
from sumy.parsers.plaintext import PlaintextParser
from sumy.nlp.tokenizers import Tokenizer
from sumy.summarizers.lsa import LsaSummarizer
import string
import os

In [ ]:


# Predefined text inputs
predefined_texts = {
    "Sample Text 1": "This is a sample text to demonstrate the text analysis capabilities of this tool. Feel free to modify this text and see the results.",
    "Sample Text 2": "Another example text to showcase the functionality of the text analysis tool. Experiment with different texts to see how the analysis changes.",
    "Sample Text 3": "Text analysis tools can provide insights into sentiment, keywords, and summaries. Try using different texts to explore these features."
}

# Text Summarization Function using Sumy
def summarize_text(text, sentence_count=3):
    parser = PlaintextParser.from_string(text, Tokenizer("english"))
    summarizer = LsaSummarizer()
    summary = summarizer(parser.document, sentence_count)
    return " ".join([str(sentence) for sentence in summary])

# Preprocess Text Function
def preprocess_text(text):
    text = text.lower()  # Lowercase text
    text = text.translate(str.maketrans('', '', string.punctuation))  # Remove punctuation
    text = " ".join([word for word in text.split() if word not in stopwords.words('english')])  # Remove stopwords
    return text

# Sentiment Analysis Function
def sentiment_analysis(text):
    blob = TextBlob(text)
    return blob.sentiment.polarity

# Generate Word Cloud Function
def generate_wordcloud(text):
    stop_words = set(stopwords.words('english'))
    wordcloud = WordCloud(stopwords=stop_words, background_color="white", width=800, height=400).generate(text)
    wordcloud_path = "wordcloud.png"
    wordcloud.to_file(wordcloud_path)
    return wordcloud_path

# Extract Keywords Function
def extract_keywords(text):
    vectorizer = TfidfVectorizer(stop_words='english')
    X = vectorizer.fit_transform([text])
    indices = X[0].toarray().argsort()[0, -10:][::-1]
    features = vectorizer.get_feature_names_out()
    keywords = [features[i] for i in indices]
    return keywords

# Function to handle the analysis
def analyze_text(selected_text, sentence_count):
    try:
        # Get the selected predefined text
        text = predefined_texts[selected_text]

        # Preprocess the text
        processed_text = preprocess_text(text)

        # Sentiment Analysis
        sentiment = sentiment_analysis(processed_text)
        sentiment_result = ""
        if sentiment > 0:
            sentiment_result = f"Positive Sentiment 🙂 (Score: {sentiment:.2f})"
        elif sentiment < 0:
            sentiment_result = f"Negative Sentiment 😟 (Score: {sentiment:.2f})"
        else:
            sentiment_result = f"Neutral Sentiment 😐 (Score: {sentiment:.2f})"

        # Word Cloud
        wordcloud_path = generate_wordcloud(processed_text)

        # Keyword Extraction
        keywords = extract_keywords(processed_text)
        keywords_result = ", ".join(keywords)

        # Text Summarization
        summary = summarize_text(text, sentence_count=sentence_count)

        return sentiment_result, wordcloud_path, keywords_result, summary

    except Exception as e:
        return f"An error occurred during analysis: {e}", None, None, None

# Gradio Interface
iface = gr.Interface(
    fn=analyze_text,
    inputs=[
        gr.components.Dropdown(
            choices=list(predefined_texts.keys()),
            label="Select Predefined Text",
            value="Sample Text 1"
        ),
        gr.components.Slider(minimum=1, maximum=10, value=3, step=1, label="🔢 Number of Summary Sentences")
    ],
    outputs=[
        gr.components.Textbox(label="📊 Sentiment Analysis"),
        gr.components.Image(type="filepath", label="🌥️ Word Cloud"),
        gr.components.Textbox(label="🔑 Top Keywords"),
        gr.components.Textbox(label="📝 Text Summary")
    ],
    title="Linguistic Lens 🔍",
    description="A Text Analysis Tool 🛠️"
)

iface.launch(share=True)
